In [1]:
#| default_exp article

In [2]:
#| export
from dataclasses import dataclass, asdict
from datetime import datetime, date
from typing import Optional, Dict, Any, List
from pathlib import Path
from pymongo.database import Database

In [3]:
from seo_rat.utils import *

In [4]:
#| export
@dataclass
class Article:
    """Class representing an article's metadata and analysis"""

    # File information (only required fields)
    relative_path: str  # Path relative to content directory
    website_domain: str
    base_path: str

    # Frontmatter data (will be populated from file)
    title: Optional[str] = None
    publishDate: Optional[str] = None
    excerpt: Optional[str] = None
    image: Optional[str] = None
    tags: Optional[List[str]] = None
    # Focus keyword
    focus_keyword: Optional[str] = None
    related_keywords: List[str] = None

    # Analysis data (will be populated from content)
    headers: List[Dict[str, Any]] = None
    internal_links: Dict[str, Dict] = None
    external_links: Dict[str, Dict] = None
    images_data: List[Dict[str, str]] = None
    phone_numbers: List[str] = None

    def __post_init__(self):
        """Load and analyze file content"""
        self.load_from_file()

    def get_full_path(self) -> str:
        """Get full file path"""
        return str(Path(self.base_path) / self.relative_path)

    def load_from_file(self):
        """Load and parse file content"""
        with open(self.get_full_path(), "r") as f:
            content = f.read()
        # Extract frontmatter
        self.title, self.publishDate, self.excerpt, self.image, self.tags = (
            extract_frontmatter(content)
        )

        # Get content without frontmatter
        main_content = exclude_front_matter(content)

        # Analyze content
        self.analyze_content(main_content)

    def analyze_content(self, content: str):
        """Analyze the content and populate metadata"""
        self.headers = get_heads_info(self.get_full_path())  # Changed from file_path

        urls_data = extract_markdown_urls_with_tags(content)
        self.internal_links = {
            url: data
            for url, data in urls_data.items()
            if url in get_internal_urls([url], self.website_domain)
        }
        self.external_links = {
            url: data
            for url, data in urls_data.items()
            if url in get_external_urls([url], self.website_domain)
        }
        self.images_data = extract_markdown_images(content)
        self.phone_numbers = detect_numbers(content)

    def to_dict(self) -> Dict[str, Any]:
        """Convert article data to MongoDB document format"""
        # Convert publishDate to ISO format string if it's a datetime
        publish_date = self.publishDate
        if isinstance(publish_date, (datetime, date)):
            publish_date = publish_date.isoformat()

        return {
            "relative_path": self.relative_path,  # Changed from file_path
            "base_path": self.base_path,
            "website_domain": self.website_domain,
            "focus_keyword": self.focus_keyword,
            "metadata": {
                "title": self.title,
                "publishDate": publish_date,  # Use converted date
                "excerpt": self.excerpt,
                "image": self.image,
                "tags": self.tags,
            },
            "analysis": {
                "headers": self.headers,
                "internal_links": self.internal_links,
                "external_links": self.external_links,
                "images": self.images_data,
                "phone_numbers": self.phone_numbers,
            },
        }

In [5]:
#| export
from difflib import SequenceMatcher
from pathlib import Path
import glob
import re


class ArticleStore:
    """Handle MongoDB operations for Article objects"""

    def __init__(self, db: Database, content_base_path: str):
        self.db = db
        self.collection = self.db.articles
        self.content_base_path = Path(content_base_path)

    def check_duplicate_before_insert(
        self, article: Article, similarity_threshold: float = 0.8
    ) -> Dict[str, Any]:
        """Check if article content is duplicate before insertion"""
        current_content = exclude_front_matter(
            open(article.get_full_path(), "r").read()
        )
        current_content = re.sub(r"\s+", " ", current_content).strip()

        # Check existing articles
        existing_articles = self.collection.find()
        duplicates = []

        for existing in existing_articles:
            try:
                # Skip if it's the same article (same website and path)
                if (
                    existing.get("website_domain") == article.website_domain
                    and existing.get("relative_path") == article.relative_path
                ):
                    continue

                # Get existing article content
                if "relative_path" in existing:
                    existing_path = self.get_full_path(existing["relative_path"])
                else:
                    existing_path = existing["file_path"]

                existing_content = exclude_front_matter(open(existing_path, "r").read())
                existing_content = re.sub(r"\s+", " ", existing_content).strip()

                # Calculate similarity
                similarity = SequenceMatcher(
                    None, current_content, existing_content
                ).ratio()

                if similarity >= similarity_threshold:
                    duplicates.append(
                        {
                            "title": existing["metadata"]["title"],
                            "similarity": similarity,
                            "path": existing.get("relative_path")
                            or existing.get("file_path"),
                        }
                    )

            except Exception as e:
                print(
                    f"Warning: Error checking article {existing.get('relative_path', 'unknown')}: {e}"
                )
                continue

        return {"has_duplicates": bool(duplicates), "duplicates": duplicates}

    def insert_or_update_article(self, article: Article) -> str:
        """Insert article if not exists, or update if exists based on website and path"""
        # First check for duplicates
        duplicate_check = self.check_duplicate_before_insert(article)
        if duplicate_check["has_duplicates"]:
            print("Warning: Similar content detected:")
            for dup in duplicate_check["duplicates"]:
                print(f"- {dup['title']} (similarity: {dup['similarity']:.2f})")

        article_dict = article.to_dict()

        # Try to update existing article
        result = self.collection.update_one(
            {
                "website_domain": article.website_domain,
                "relative_path": article.relative_path,
            },
            {"$set": article_dict},
            upsert=True,
        )

        if result.upserted_id:
            return str(result.upserted_id)
        else:
            # Get the _id of the existing document
            existing = self.collection.find_one(
                {
                    "website_domain": article.website_domain,
                    "relative_path": article.relative_path,
                }
            )
            return str(existing["_id"])

    def get_all_article_files(self) -> List[str]:
        """Get all markdown files in content directory"""
        pattern = str(self.content_base_path / "**/*.md")
        files = glob.glob(pattern, recursive=True)
        return [str(Path(f).relative_to(self.content_base_path)) for f in files]

    def find_by_tag(self, tag: str) -> List[Dict]:
        """Find articles by tag"""
        return list(self.collection.find({"metadata.tags": tag}))

    def find_articles_with_phone_numbers(self) -> List[Dict]:
        """Find articles containing phone numbers"""
        return list(
            self.collection.find(
                {"analysis.phone_numbers": {"$exists": True, "$ne": []}}
            )
        )

    def get_full_path(self, relative_path: str) -> str:
        """Convert relative path to full path"""
        return str(self.content_base_path / relative_path)

    def find_by_domain(self, domain: str) -> List[Dict]:
        """Find all articles for a specific website domain"""
        return list(self.collection.find({"website_domain": domain}))

    def search_in_title(self, keyword: str) -> List[Dict]:
        """Search for keyword in article titles"""
        return list(
            self.collection.find(
                {"metadata.title": {"$regex": keyword, "$options": "i"}}
            )
        )

    def set_article_keywords(
        self, relative_path: str, focus_keyword: str, related_keywords: List[str] = None
    ) -> bool:
        """Set focus keyword and related keywords for an article"""
        if related_keywords is None:
            related_keywords = []
        result = self.collection.update_one(
            {"relative_path": relative_path},
            {
                "$set": {
                    "focus_keyword": focus_keyword,
                    "related_keywords": related_keywords,
                }
            },
        )
        return result.modified_count > 0

    def get_article_metadata(self, relative_path: str) -> Dict:
        """Get complete metadata for a specific article"""
        article = self.collection.find_one({"relative_path": relative_path})
        if article is None:
            raise ValueError(f"No article found with relative_path: {relative_path}")
        return article

    def get_article_analysis(self, relative_path: str) -> Dict:
        """Get just the analysis part of an article"""
        article = self.get_article_metadata(relative_path)
        return article.get("analysis", {})

    def get_article_headers(self, relative_path: str) -> List[Dict]:
        """Get headers from specific article"""
        analysis = self.get_article_analysis(relative_path)
        return analysis.get("headers", [])

    def get_article_links(self, relative_path: str) -> Dict:
        """Get all links from specific article"""
        analysis = self.get_article_analysis(relative_path)
        return {
            "internal": analysis.get("internal_links", {}),
            "external": analysis.get("external_links", {}),
        }

    def get_article_phone_numbers(self, relative_path: str) -> List[str]:
        """Get phone numbers from specific article"""
        analysis = self.get_article_analysis(relative_path)
        return analysis.get("phone_numbers", [])

    def get_article_images(self, relative_path: str) -> List[Dict[str, str]]:
        """Get images data from specific article"""
        analysis = self.get_article_analysis(relative_path)
        return analysis.get("images", [])

    def analyze_header_hierarchy(self, relative_path: str) -> Dict[str, List[Dict]]:
        """Analyze headers by level"""
        article = self.get_article_metadata(relative_path)
        headers = article["analysis"]["headers"]
        return {
            "h1": [h for h in headers if h["type"] == "h1"],
            "h2": [h for h in headers if h["type"] == "h2"],
            "h3": [h for h in headers if h["type"] == "h3"],
            "h4": [h for h in headers if h["type"] == "h4"],
            "h5": [h for h in headers if h["type"] == "h5"],
            "h6": [h for h in headers if h["type"] == "h6"],
        }

    def calculate_keyword_density(self, relative_path: str) -> Dict[str, float]:
        """Calculate keyword density for focus and related keywords"""
        article = self.get_article_metadata(relative_path)
        keywords = self.get_article_keywords(relative_path)
        full_path = self.get_full_path(relative_path)
        content = exclude_front_matter(open(full_path, "r").read())

        # Convert content to lowercase for case-insensitive matching
        content_lower = content.lower()
        # Get total word count
        total_words = len(content.split())

        densities = {}

        # Calculate focus keyword density
        if keywords.get("focus_keyword"):
            focus_kw = keywords["focus_keyword"].lower()
            focus_count = content_lower.count(focus_kw)
            densities["focus_keyword"] = {
                "keyword": keywords["focus_keyword"],
                "count": focus_count,
                "density": (focus_count / total_words) * 100 if total_words > 0 else 0,
            }

        # Calculate related keywords density
        densities["related_keywords"] = []
        for kw in keywords.get("related_keywords", []):
            kw_lower = kw.lower()
            count = content_lower.count(kw_lower)
            densities["related_keywords"].append(
                {
                    "keyword": kw,
                    "count": count,
                    "density": (count / total_words) * 100 if total_words > 0 else 0,
                }
            )
        self.collection.update_one(
            {"relative_path": relative_path}, {"$set": {"keyword_density": densities}}
        )
        return densities

    def get_article_keywords(self, relative_path: str) -> Dict[str, Any]:
        """Get keywords data for an article"""
        article = self.get_article_metadata(relative_path)
        return {
            "focus_keyword": article.get("focus_keyword"),
            "related_keywords": article.get("related_keywords", []),
        }

    def analyze_seo_rules(self, relative_path: str) -> Dict[str, Any]:
        """Analyze SEO rules for a specific article"""
        article = self.get_article_metadata(relative_path)
        keywords = self.get_article_keywords(relative_path)
        full_path = self.get_full_path(relative_path)
        focus_keyword = keywords.get("focus_keyword")
        related_keywords = keywords.get("related_keywords", [])

        # Get all headers of type h1
        h1_headers = [h for h in article["analysis"]["headers"] if h["type"] == "h1"]

        # Check if focus keyword is in excerpt
        excerpt = article["metadata"].get("excerpt", "")

        # Get content for 10% analysis
        content = exclude_front_matter(open(full_path, "r").read())
        first_10_percent = content[: int(len(content) * 0.1)]

        # Get image alt texts
        alt_texts = [img["alt_text"] for img in article["analysis"]["images"]]

        # Check if focus keyword is unique across all articles
        other_articles = list(
            self.collection.find(
                {
                    "relative_path": {"$ne": relative_path},
                    "focus_keyword": focus_keyword,
                }
            )
        )

        # Count and find positions of focus keyword
        keyword_positions = []
        content_lower = content.lower()
        focus_keyword_lower = focus_keyword.lower() if focus_keyword else ""
        pos = 0
        while (
            focus_keyword
            and (pos := content_lower.find(focus_keyword_lower, pos)) != -1
        ):
            keyword_positions.append(pos)
            pos += 1

        analysis = {
            "h1_count": len(h1_headers),
            "has_single_h1": len(h1_headers) == 1,
            "focus_keyword_in_excerpt": (
                focus_keyword and focus_keyword.lower() in excerpt.lower()
                if excerpt
                else False
            ),
            "keywords_in_first_10_percent": {
                "focus_keyword": focus_keyword
                and focus_keyword.lower() in first_10_percent.lower(),
                "related_keywords": [
                    kw
                    for kw in related_keywords
                    if kw.lower() in first_10_percent.lower()
                ],
            },
            "keywords_in_alt_text": {
                "focus_keyword": any(
                    focus_keyword and focus_keyword.lower() in alt.lower()
                    for alt in alt_texts
                ),
                "related_keywords": [
                    kw
                    for kw in related_keywords
                    if any(kw.lower() in alt.lower() for alt in alt_texts)
                ],
            },
            "has_external_links": bool(article["analysis"]["external_links"]),
            "focus_keyword_is_unique": len(other_articles) == 0,  # Changed this line
            "focus_keyword_stats": {
                "count": len(keyword_positions),
                "positions": keyword_positions,
            },
        }

        # Update article with analysis
        self.collection.update_one(
            {"relative_path": relative_path}, {"$set": {"seo_analysis": analysis}}
        )
        return analysis

    def detect_duplicate_content(
        self, relative_path: str, similarity_threshold: float = 0.8
    ) -> Dict[str, Any]:
        """Detect similar content across articles"""
        current_article = self.get_article_metadata(relative_path)
        full_path = self.get_full_path(relative_path)
        current_content = exclude_front_matter(open(full_path, "r").read())

        # Get all other articles - check both old and new path fields
        similar_articles = []
        all_articles = self.collection.find(
            {
                "$or": [
                    {"relative_path": {"$ne": relative_path}},
                    {"file_path": {"$exists": True}},  # For backwards compatibility
                ]
            }
        )

        for article in all_articles:
            # Handle both old and new path formats
            try:
                if "relative_path" in article:
                    other_path = article["relative_path"]
                else:
                    # Skip if it's the same article
                    if article["file_path"] == full_path:
                        continue
                    # Convert old file_path to relative_path
                    other_path = str(
                        Path(article["file_path"]).relative_to(self.content_base_path)
                    )

                other_full_path = self.get_full_path(other_path)
                other_content = exclude_front_matter(open(other_full_path, "r").read())

                # Calculate similarity ratio
                similarity = SequenceMatcher(
                    None, current_content, other_content
                ).ratio()

                if similarity >= similarity_threshold:
                    similar_articles.append(
                        {
                            "relative_path": other_path,
                            "similarity": similarity,
                            "title": article["metadata"]["title"],
                        }
                    )

            except (KeyError, ValueError) as e:
                print(f"Warning: Skipping article due to path error: {e}")
                continue

        result = {
            "has_duplicates": len(similar_articles) > 0,
            "similar_articles": similar_articles,
        }

        self.collection.update_one(
            {"relative_path": relative_path},
            {"$set": {"duplicate_content_analysis": result}},
        )

        return result

    def analyze_keyword_cannibalization(self, focus_keyword: str) -> Dict[str, Any]:
        """Detect keyword cannibalization across articles"""
        # Find all articles using this focus keyword
        articles = list(
            self.collection.find(
                {
                    "$or": [
                        {"focus_keyword": focus_keyword},
                        {"related_keywords": focus_keyword},
                    ]
                }
            )
        )

        if len(articles) <= 1:
            return {
                "has_cannibalization": False,
                "keyword": focus_keyword,
                "count": len(articles),
            }

        cannibalization_data = {
            "has_cannibalization": True,
            "keyword": focus_keyword,
            "count": len(articles),
            "articles": [],
        }

        for article in articles:
            try:
                # Handle both old and new path formats
                if "relative_path" in article:
                    article_path = article["relative_path"]
                else:
                    # Convert old file_path to relative_path
                    article_path = str(
                        Path(article["file_path"]).relative_to(self.content_base_path)
                    )

                density = self.calculate_keyword_density(article_path)

                article_data = {
                    "relative_path": article_path,
                    "title": article["metadata"]["title"],
                    "publish_date": article["metadata"]["publishDate"],
                    "keyword_type": (
                        "focus"
                        if article.get("focus_keyword") == focus_keyword
                        else "related"
                    ),
                    "density": (
                        density["focus_keyword"]["density"]
                        if article.get("focus_keyword") == focus_keyword
                        else next(
                            (
                                kw["density"]
                                for kw in density["related_keywords"]
                                if kw["keyword"] == focus_keyword
                            ),
                            0,
                        )
                    ),
                }
                cannibalization_data["articles"].append(article_data)

            except (KeyError, ValueError) as e:
                print(
                    f"Warning: Skipping article in cannibalization analysis due to error: {e}"
                )
                continue

        return cannibalization_data
    def analyze_content_groups(self, similarity_threshold: float = 0.8) -> Dict[str, Any]:
        """Analyze and group similar articles"""
        all_articles = list(self.collection.find())
        content_groups = []
        processed = set()
        
        for article in all_articles:
            article_path = article.get('relative_path')
            if article_path in processed:
                continue
                
            # Start a new group
            group = {
                'main_article': {
                    'title': article['metadata']['title'],
                    'path': article_path
                },
                'similar_articles': [],
                'common_content': [],
                'unique_content': []
            }
            
            # Get content of main article
            main_content = exclude_front_matter(open(self.get_full_path(article_path), 'r').read())
            main_content_lines = main_content.split('\n')
            
            for other in all_articles:
                if other.get('relative_path') == article_path:
                    continue
                    
                other_path = other.get('relative_path')
                other_content = exclude_front_matter(open(self.get_full_path(other_path), 'r').read())
                
                # Calculate similarity
                similarity = SequenceMatcher(None, main_content, other_content).ratio()
                
                if similarity >= similarity_threshold:
                    processed.add(other_path)
                    group['similar_articles'].append({
                        'title': other['metadata']['title'],
                        'path': other_path,
                        'similarity': similarity
                    })
                    
                    # Find common content
                    other_lines = other_content.split('\n')
                    common_lines = set()
                    for line in main_content_lines:
                        if line.strip() and line in other_lines:
                            common_lines.add(line)
                    
                    group['common_content'] = list(common_lines)
                    
                    # Find unique content in main article
                    unique_lines = set()
                    for line in main_content_lines:
                        if line.strip() and line not in other_lines:
                            unique_lines.add(line)
                    
                    group['unique_content'] = list(unique_lines)
            
            if group['similar_articles']:
                content_groups.append(group)
                processed.add(article_path)
        
        return {
            'total_articles': len(all_articles),
            'groups': content_groups,
            'unique_articles': len(all_articles) - len(processed),
            'duplicate_groups': len(content_groups)
        }
